In [5]:
import requests 
import bs4
from tqdm import tqdm_notebook as tqdm
import pandas as pd

# help function
def get_content(soup):
    ## 查找所有html 元素 抓出內容
    main_container = soup.find(id='main-container')
    # 把所有文字都抓出來
    all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
    pre_text = all_text.split('--')[0]
    # 把每段文字 根據 '\n' 切開
    texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
    contents = texts[2:]
    # 內容
    content = ''.join(contents)
    return content

def get_total_page_number(URL):
    response = requests.get(URL, headers = {'cookie': 'over18=1;'})
    soup = bs4.BeautifulSoup(response.text,"html.parser")
    number = int(str(soup.select('#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(2)')[0]).split('index')[1].split('.html')[0])
    return number


def get_this_index_data(URL):
    # 設定Header與Cookie
    my_headers = {'cookie': 'over18=1;'}
    # 發送get 請求 到 ptt 八卦版
    response = requests.get(URL, headers = my_headers)

    #  把網頁程式碼(HTML) 丟入 bs4模組分析
    soup = bs4.BeautifulSoup(response.text,"html.parser")

    ## PTT 上方4個欄位
    result = soup.find_all('div','title')
    urls = []
    titles = []
    contents = []
    for i in result:
        try:
            post_url = 'https://www.ptt.cc'+str(i).split('href="')[1].split('">')[0]
            urls.append(post_url)
            titles.append(str(i).split("]")[1].split("<")[0])
            response = requests.get(post_url, headers = my_headers)
            soup = bs4.BeautifulSoup(response.text,"html.parser")
            contents.append(get_content(soup))
        except:
            pass #(本文已被刪除)
    df = pd.DataFrame()
    df['title'] = titles
    df['url'] = urls
    df['content'] = contents
    return df

def get_some_page_ptt_data(URL,last_n_page):
    total_page_number = get_total_page_number(URL)
    URLS = [f"https://www.ptt.cc/bbs/HatePolitics/index{i}.html" for i in range(total_page_number,total_page_number-last_n_page,-1)]
    df = get_this_index_data(URL) #最新的PAGE
    for u in tqdm(URLS): #最近N個PAGE
        d = get_this_index_data(u)
        df = pd.concat([df,d]) #串起來
    return df



# 爬蟲抓取資料

In [8]:
URL = "https://www.ptt.cc/bbs/HatePolitics/index.html"
last_n_page = 30
df = get_some_page_ptt_data(URL,last_n_page)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6500\306101296.py:63: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for u in tqdm(URLS): #最近N個PAGE


  0%|          | 0/30 [00:00<?, ?it/s]

# 保存

In [21]:
df.to_excel('ptt_post.xlsx')
df

,title,url,content
0,紐時：FBI介入 美國加強打擊中國海外警,https://www.ptt.cc/bbs/HatePolitics/M.16735664...,1.新聞網址︰※ 網址超過一行 請縮網址 ※https://bit.ly/3GCghpg2....
1,邱顯智 FB,https://www.ptt.cc/bbs/HatePolitics/M.16735665...,1.轉錄網址︰https://bit.ly/3vYUHq1※ 網址超過一行 請縮網址 ※2....
2,林余案筆記,https://www.ptt.cc/bbs/HatePolitics/M.16735666...,昨天，完整答辯書已經出來，https://tinyurl.com/yztu9zm3關鍵1.2...
3,政治黑特板板規__20220803,https://www.ptt.cc/bbs/HatePolitics/M.16595039...,╭──────╮│第一章 總則│╰──────╯￣￣￣￣￣￣￣￣1.本板接受中華民國政府頒布...
4,請大家發文前 多注意板規,https://www.ptt.cc/bbs/HatePolitics/M.16725574...,餓死抬頭請大家發文前多注意一下板規文章分類內文之前就有設定範本 參考尤其最多人違規的 發文相...
...,...,...,...
15,為什麼不一開始處理余正煌？,https://www.ptt.cc/bbs/HatePolitics/M.16734473...,套句電影臺詞，兩種東西沒價1是正義，2是老子不願意余這不好收買，首先就是你收買他，但塔綠斑們...
16,就算小智有抄襲 但沒有違反著作權 怎解?,https://www.ptt.cc/bbs/HatePolitics/M.16734474...,現在小智請來著作權大律師上場 大律師有多大?就連法官在學期間讀的書 說不定都是那位大律師寫的...
17,為什麼林耕仁沒有出來戰清白？,https://www.ptt.cc/bbs/HatePolitics/M.16734474...,眾所周知 在跨年前林耕仁的陽明交通大學碩士論文也被撤銷了 林耕仁就淡淡的回尊重學倫會審查結果...
18,論文出包還有許淑華！修正這天到期 逢甲,https://www.ptt.cc/bbs/HatePolitics/M.16734476...,論文出包還有許淑華！修正這天到期 逢甲：若沒收到撤學位〔記者蘇孟娟／台中報導〕前新竹市長林智...


In [22]:
df['all_text'] = df['title']+df['content']
df

,title,url,content,all_text
0,紐時：FBI介入 美國加強打擊中國海外警,https://www.ptt.cc/bbs/HatePolitics/M.16735664...,1.新聞網址︰※ 網址超過一行 請縮網址 ※https://bit.ly/3GCghpg2....,紐時：FBI介入 美國加強打擊中國海外警1.新聞網址︰※ 網址超過一行 請縮網址 ※htt...
1,邱顯智 FB,https://www.ptt.cc/bbs/HatePolitics/M.16735665...,1.轉錄網址︰https://bit.ly/3vYUHq1※ 網址超過一行 請縮網址 ※2....,邱顯智 FB1.轉錄網址︰https://bit.ly/3vYUHq1※ 網址超過一行 請...
2,林余案筆記,https://www.ptt.cc/bbs/HatePolitics/M.16735666...,昨天，完整答辯書已經出來，https://tinyurl.com/yztu9zm3關鍵1.2...,林余案筆記昨天，完整答辯書已經出來，https://tinyurl.com/yztu9zm...
3,政治黑特板板規__20220803,https://www.ptt.cc/bbs/HatePolitics/M.16595039...,╭──────╮│第一章 總則│╰──────╯￣￣￣￣￣￣￣￣1.本板接受中華民國政府頒布...,政治黑特板板規__20220803╭──────╮│第一章 總則│╰──────╯￣￣￣￣...
4,請大家發文前 多注意板規,https://www.ptt.cc/bbs/HatePolitics/M.16725574...,餓死抬頭請大家發文前多注意一下板規文章分類內文之前就有設定範本 參考尤其最多人違規的 發文相...,請大家發文前 多注意板規餓死抬頭請大家發文前多注意一下板規文章分類內文之前就有設定範本 參...
...,...,...,...,...
15,為什麼不一開始處理余正煌？,https://www.ptt.cc/bbs/HatePolitics/M.16734473...,套句電影臺詞，兩種東西沒價1是正義，2是老子不願意余這不好收買，首先就是你收買他，但塔綠斑們...,為什麼不一開始處理余正煌？套句電影臺詞，兩種東西沒價1是正義，2是老子不願意余這不好收買，...
16,就算小智有抄襲 但沒有違反著作權 怎解?,https://www.ptt.cc/bbs/HatePolitics/M.16734474...,現在小智請來著作權大律師上場 大律師有多大?就連法官在學期間讀的書 說不定都是那位大律師寫的...,就算小智有抄襲 但沒有違反著作權 怎解?現在小智請來著作權大律師上場 大律師有多大?就連法...
17,為什麼林耕仁沒有出來戰清白？,https://www.ptt.cc/bbs/HatePolitics/M.16734474...,眾所周知 在跨年前林耕仁的陽明交通大學碩士論文也被撤銷了 林耕仁就淡淡的回尊重學倫會審查結果...,為什麼林耕仁沒有出來戰清白？眾所周知 在跨年前林耕仁的陽明交通大學碩士論文也被撤銷了 林耕...
18,論文出包還有許淑華！修正這天到期 逢甲,https://www.ptt.cc/bbs/HatePolitics/M.16734476...,論文出包還有許淑華！修正這天到期 逢甲：若沒收到撤學位〔記者蘇孟娟／台中報導〕前新竹市長林智...,論文出包還有許淑華！修正這天到期 逢甲論文出包還有許淑華！修正這天到期 逢甲：若沒收到撤學...


In [63]:
def get_score_by_person(df,person_name):
    idx_lst = []
    for idx,text in enumerate(df['all_text']):
        if person_name in text:
            idx_lst.append(idx)
    key_df = df.iloc[idx_lst,:]
    key_df = key_df.reset_index(drop=True)
    print(len(key_df))
    key_df['情緒'] = 0
    for idx,text in tqdm(enumerate(key_df['all_text'].values.tolist())):
        key_df.loc[idx,'情緒'] = classifier(text, candidate_labels)['labels'][0]
    score = (key_df['情緒']=='positive').sum()/(key_df['情緒']=='negative').sum()
    return score

In [29]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", device=0) #GPU
candidate_labels = ["positive", "negative"]
text = "I don't know why I like this movie so well, but I never get tired of watching it."
classifier(text, candidate_labels)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'sequence': "I don't know why I like this movie so well, but I never get tired of watching it.",
 'labels': ['positive', 'negative'],
 'scores': [0.8987417221069336, 0.101258285343647]}

In [64]:
get_score_by_person(df,person_name='蔡英文')

39


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6500\684795557.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx,text in tqdm(enumerate(key_df['all_text'].values.tolist())):


0it [00:00, ?it/s]

c:\Users\Administrator\anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.95

In [65]:
get_score_by_person(df,person_name='林智堅')

68


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6500\684795557.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx,text in tqdm(enumerate(key_df['all_text'].values.tolist())):


0it [00:00, ?it/s]

c:\Users\Administrator\anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.5111111111111111

In [66]:
get_score_by_person(df,person_name='陳時中')

11


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6500\684795557.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx,text in tqdm(enumerate(key_df['all_text'].values.tolist())):


0it [00:00, ?it/s]

c:\Users\Administrator\anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.375